In [1]:
import json
import logging
import numpy as np
logger = logging.getLogger(__name__)

In [2]:
class QwenDatasets:
    def __init__(self, datapath="../data/qwen-bailian-usagetraces-anon"):
        with open(datapath+"/qwen_traceA_blksz_16.jsonl", "r") as f:
            self.trace_a = f.readlines()
        self.trace_a_size = len(self.trace_a)
        
        with open(datapath+"/qwen_traceB_blksz_16.jsonl", "r") as f:
            self.trace_b = f.readlines()
        self.trace_b_size = len(self.trace_b)
        
        logger.warning(f"loading trace A/B from {datapath}, trace_a_size={self.trace_a_size}, trace_b_size={self.trace_b_size}")
    
    def trace_a(self, idx = None):
        if idx is None:
            idx = 0
            logger.info()
        return self.trace_a
        
    def trace_b(self):
        return self.trace_b

In [4]:
import radix_cache as rc

qwen_data = QwenDatasets()

def radix_with_qwen_simulation(data):
    def line_parser(line: str):
        sample = json.loads(line)
        return sample["hash_ids"], sample["timestamp"]
        
    page_size = 1 # already given block hash, no need to convert
    timestamp = 0
    tree = rc.RadixCache(page_size=page_size, disable=False, timestamp=timestamp)
    max_timestamp = - 1
    for line in data:
        hash_ids, timestamp = line_parser(line)
        tree.match_prefix(key=hash_ids, timestamp=timestamp)
        tree.insert(key=hash_ids, timestamp=timestamp)
        max_timestamp = max(max_timestamp, timestamp)

    # tree.pretty_print()
    tree.update_cache_status(cur_time=max_timestamp)
    tree.show_metrics()

trace_a = qwen_data.trace_a
radix_with_qwen_simulation(trace_a)

trace_b = qwen_data.trace_b
radix_with_qwen_simulation(trace_b)

loading trace A/B from ../data/qwen-bailian-usagetraces-anon, trace_a_size=43058, trace_b_size=172800


num_queries = 6293976
num_hits = 117914
hit_rate = 1.87%
num_actives = 1196596
num_stores = 2656400
active_rate = 45.05%
retrieve_time_list = [-1758625135.102928, 7197.588, 7197.392, 7189.528, 2041.748, 7186.025, 0.0, 146.676, 7018.235000000001, 6491.321]
max_retrieve_time = 7199.138
store_duration_list = [-1758625134.999928, 7199.258, 7197.558, 7196.861, 7192.4619999999995, 7186.025, 7181.001, 7180.755, 7174.341, 7167.7119999999995]
max_store_duration = 7199.258
num_queries = 9956137
num_hits = 330756
hit_rate = 3.32%
num_actives = 130770
num_stores = 4638378
active_rate = 2.82%
retrieve_time_list = [-1758625137.247654, 6880.29, 7152.478999999999, 7196.671, 7197.084, 7188.118, 7199.458, 3551.297, 7031.696999999999, 0.0]
max_retrieve_time = 7199.458
store_duration_list = [-1758625137.184654, 7199.963000000001, 7199.95, 7199.934, 7199.917, 7199.832, 7199.776, 7199.711, 7199.611, 7199.586]
max_store_duration = 7199.963000000001
